In [2]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from tabulate import tabulate
import pickle

import os
import sys
sys.path.append(os.path.abspath(os.curdir))

from StarV.set.imagestar import ImageStar
from StarV.set.sparseimagestar2dcoo import SparseImageStar2DCOO
from StarV.set.sparseimagestar2dcsr import SparseImageStar2DCSR
from StarV.verifier.certifier import certifyRobustness_pixel
from StarV.util.load import *
from sklearn.metrics import jaccard_score


from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

np.set_printoptions(edgeitems=30, linewidth=100000)

def pixel_attack(image, format='IM', data_type='mnist', N_max=20, br=150, delta=0.001):
    """
        @N_max: maximum allowable number of attacked pixels
        @delta: size of input sets
        @br: threshold to set pixel to 0
    """
    assert N_max <= np.prod(image.shape), f"number of maximum attacked pixels shouldn't be greater than number of image pixels"
    assert format in ['IM', 'SIM_coo', 'SIM_csr'], f"return_class should be \'IM\', \'SIM_coo\', \'SIM_csr\', but received {format}"
    
    if data_type == 'mnist':
        h = image.shape[0] - 1
        w = image.shape[1] - 1
    else:
        h, w = 63, 83
        
    attack_image = image.copy() 
    cnt = 0
    for i in range(h):
        for j in range(w):
            if image[i, j] > br:
                attack_image[i, j] = 0
                cnt += 1
                if cnt == N_max:
                    break
        else:
             continue
        break
        
    noise = attack_image - image
    V = np.concatenate([image[:, :, None, None], noise[:, :, None, None]], axis=3)
    C = np.array([[1], [-1]])
    d = np.array([1, delta-1])
    IM = ImageStar(V, C, d, np.array([1-delta]), np.array([1]))
    if format == 'IM':
        return IM
    elif format == 'SIM_coo':
        return IM.toSIM('coo')
    else:
        return IM.toSIM('csr')
    
# def attack_multiple_images(images, format='IM', data_type='mnist', N_max=[20], br=150, delta=0.001):
# #     @N_max: a list containing maximum allowable number of attacked pixels
# #     @delta: sizes of input sets
# #     @br: threshold to set pixel to 0

#     assert isinstance(N_max, list), f"N_max should be a list containing sizes of input sets, but received {type(N_max)}"
#     outputs = []
#     for n_max in range(len(N_max)):
#         out_nmax = []
#         for i in range(images.shape[2]):
#             out_nmax.append(pixel_attack(images[:, :, i] , format, data_type, n_max, br, delta))
#         outputs.append(out_nmax)
#     return outputs

def attack_multiple_images(images, format='IM', data_type='mnist', N_max=[20], br=150, delta=0.001):
#     @N_max: a list containing maximum allowable number of attacked pixels
#     @delta: sizes of input sets
#     @br: threshold to set pixel to 0

    assert isinstance(N_max, list), f"N_max should be a list containing sizes of input sets, but received {type(N_max)}"
    outputs = []
    for n_max in range(len(N_max)):
        out_nmax = []
        for image in images:
            out_nmax.append(pixel_attack(image, format, data_type, n_max, br, delta))
        outputs.append(out_nmax)
    return outputs
            
def load_dataset(folder_dir, data_type, dtype='float64'):
    if data_type == 'mnist':
        mat_file = scipy.io.loadmat(f"{folder_dir}/test_images.mat")
    else:
        mat_file = scipy.io.loadmat(f"{folder_dir}/m2nist_6484_test_images.mat")
    data = mat_file['im_data'].astype(dtype)
    labels = mat_file['im_labels'].ravel() - 1
    return data, labels

def load_network(folder_dir, net_type='relu', data_type='mnist', dtype='float64'):
    assert data_type in ['mnist', 'm2nist'], f"data_type should be either 'mnist' or 'm2nist', but received {data_type}"
    if data_type == 'mnist':
        assert net_type in ['dilated', 'relu_maxpool', 'relu']
        if net_type == 'dilated':
            net_dir = f"{folder_dir}/mnist_{net_type}_net_21_later_83iou.onnx"
        else:
            net_dir = f"{folder_dir}/net_mnist_3_{net_type}.onnx"
    else:
        assert net_type in ['dilatedcnn_avgpool', 'transposedcnn_avgpool', 'dilated']
        if net_type == 'dilated':
            net_dir = f"{folder_dir}/m2nist_{net_type}_72iou_24layer.onnx"
        elif net_type == 'dilatedcnn_avgpool':
            net_dir = f"{folder_dir}/m2nist_62iou_{net_type}.onnx"
        else:
            net_dir = f"{folder_dir}/m2nist_75iou_{net_type}.onnx"
            
    return load_onnx_network(net_dir, num_pixel_classes=10, dtype=dtype)

def verify_CAV2021_MNIST_SSNN(net_type, dtype='float64'):
    print('=========================================================================================')
    print(f"Verification of CAV2021 MNIST Semantic Segmentation Neural Netowrk against Pixel Attacks")
    print('=========================================================================================\n')
    
    data_type = 'mnist'
    folder_dir = f"./SparseImageStar_evaluation/CAV2021_SSNN/MNIST"
    starvNet = load_network(folder_dir, net_type, data_type, dtype)
    
    print(starvNet.info())
    data, labels = load_dataset(folder_dir, data_type)
    N = 20 # number of tested images
    labels = labels[:N]
    images = [data[:, :, i] for i in range(N)]
    
    N_max = [10, 20, 30, 40, 50]
    M = len(N_max)
    br = 150
    delta = 0.001
    

    print(f"Verifying {net_type} SSNN with ImageStar")
    avg_numRbIM = np.zeros(M)
    avg_numUnkIM = np.zeros(M)
    avg_numMisIM = np.zeros(M)
    avg_numAttIM = np.zeros(M)
    avg_riouIM = np.zeros(M)
    avg_rvIM = np.zeros(M)
    avg_rsIM = np.zeros(M)
    avg_vtIM = np.zeros(M)
    
    IM_sets = attack_multiple_images(images, 'IM', data_type, N_max, br, delta) #returns a list in shape len(N_max) x len(images)
    
    for i in range(M):
        _, _, _, _, avg_data = certifyRobustness_pixel(net=starvNet, in_sets=IM_sets[i], in_datas=images,
            veriMethod='BFS', reachMethod='approx', lp_solver='gurobi', pool=None, 
            RF=0.0, DR=0, return_output=False, show=False)
        avg_numRbIM[i], avg_numUnkIM[i], avg_numMisIM[i], avg_numAttIM[i], avg_riouIM[i], avg_rvIM[i], avg_rsIM[i], avg_vtIM[i] = avg_data

    
    print(f"\nVerifying {net_type} SSNN with Sparse Image Star in CSR format")
    avg_numRbCSR = np.zeros(M)
    avg_numUnkCSR = np.zeros(M)
    avg_numMisCSR = np.zeros(M)
    avg_numAttCSR = np.zeros(M)
    avg_riouCSR = np.zeros(M)
    avg_rvCSR = np.zeros(M)
    avg_rsCSR = np.zeros(M)
    avg_vtCSR = np.zeros(M)
    
    CSR_sets = attack_multiple_images(images, 'SIM_csr', data_type, N_max, br, delta) #returns a list in shape len(N_max) x len(images)
    
    for i in range(M):
        _, _, _, _, avg_data = certifyRobustness_pixel(net=starvNet, in_sets=CSR_sets[i], in_datas=images,
            veriMethod='BFS', reachMethod='approx', lp_solver='gurobi', pool=None, 
            RF=0.0, DR=0, return_output=False, show=False)
        avg_numRbCSR[i], avg_numUnkCSR[i], avg_numMisCSR[i], avg_numAttCSR[i], avg_riouCSR[i], avg_rvCSR[i], avg_rsCSR[i], avg_vtCSR[i] = avg_data
        
        
    print(f"\nVerifying {net_type} SSNN with Sparse Image Star in COO format")
    avg_numRbCOO = np.zeros(M)
    avg_numUnkCOO = np.zeros(M)
    avg_numMisCOO = np.zeros(M)
    avg_numAttCOO = np.zeros(M)
    avg_riouCOO = np.zeros(M)
    avg_rvCOO = np.zeros(M)
    avg_rsCOO = np.zeros(M)
    avg_vtCOO = np.zeros(M)
    
    COO_sets = attack_multiple_images(images, 'SIM_coo', data_type, N_max, br, delta) #returns a list in shape len(N_max) x len(images)
    
    for i in range(M):
        _, _, _, _, avg_data = certifyRobustness_pixel(net=starvNet, in_sets=COO_sets[i], in_datas=images,
            veriMethod='BFS', reachMethod='approx', lp_solver='gurobi', pool=None, 
            RF=0.0, DR=0, return_output=False, show=False)
        avg_numRbCOO[i], avg_numUnkCOO[i], avg_numMisCOO[i], avg_numAttCOO[i], avg_riouCOO[i], avg_rvCOO[i], avg_rsCOO[i], avg_vtCOO[i] = avg_data
    
    IM_data = [avg_numRbIM, avg_numUnkIM, avg_numMisIM, avg_numAttIM, avg_riouIM, avg_rvIM, avg_rsIM, avg_vtIM]
    CSR_data = [avg_numRbCSR, avg_numUnkCSR, avg_numMisCSR, avg_numAttCSR, avg_riouCSR, avg_rvCSR, avg_rsCSR, avg_vtCSR]
    COO_data = [avg_numRbCOO, avg_numUnkCOO, avg_numMisCOO, avg_numAttCOO, avg_riouCOO, avg_rvCOO, avg_rsCOO, avg_vtCOO]
    
    path = f"./SparseImageStar_evaluation/results"
    save_file = path + f"/{net_type}SSNN_MNIST_CAV2021_results.pkl"
    pickle.dump([IM_data, CSR_data, COO_data], open(save_file, "wb"))
    
    print('=====================================================')
    print('DONE!')
    print('=====================================================')
    

def verify_CAV2021_M2NIST_SSNN(net_type, dtype='float64'):
    print('==========================================================================================')
    print(f"Verification of CAV2021 M2NIST Semantic Segmentation Neural Netowrk against Pixel Attacks")
    print('==========================================================================================\n')
    
    data_type = 'm2nist'
    folder_dir = f"./SparseImageStar_evaluation/CAV2021_SSNN/M2NIST"
    starvNet = load_network(folder_dir, net_type, data_type, dtype)
    
    print(starvNet.info())
    data, labels = load_dataset(folder_dir, data_type)
    N = 20 # number of tested images
    labels = labels[:N]
    images = [data[:, :, i] for i in range(N)]
    
    N_max = [10, 20, 30, 40, 50]
    M = len(N_max)
    br = 150
    delta = 0.001

    
    print(f"Verifying {net_type} SSNN with ImageStar")
    avg_numRbIM = np.zeros(M)
    avg_numUnkIM = np.zeros(M)
    avg_numMisIM = np.zeros(M)
    avg_numAttIM = np.zeros(M)
    avg_riouIM = np.zeros(M)
    avg_rvIM = np.zeros(M)
    avg_rsIM = np.zeros(M)
    avg_vtIM = np.zeros(M)
    
    IM_sets = attack_multiple_images(images, 'IM', data_type, N_max, br, delta) #returns a list in shape len(N_max) x len(images)
    
    for i in range(M):
        _, _, _, _, avg_data = certifyRobustness_pixel(net=starvNet, in_sets=IM_sets[i], in_datas=images,
            veriMethod='BFS', reachMethod='approx', lp_solver='gurobi', pool=None, 
            RF=0.0, DR=0, return_output=False, show=False)
        avg_numRbIM[i], avg_numUnkIM[i], avg_numMisIM[i], avg_numAttIM[i], avg_riouIM[i], avg_rvIM[i], avg_rsIM[i], avg_vtIM[i] = avg_data

    
    print(f"\nVerifying {net_type} SSNN with Sparse Image Star in CSR format")
    avg_numRbCSR = np.zeros(M)
    avg_numUnkCSR = np.zeros(M)
    avg_numMisCSR = np.zeros(M)
    avg_numAttCSR = np.zeros(M)
    avg_riouCSR = np.zeros(M)
    avg_rvCSR = np.zeros(M)
    avg_rsCSR = np.zeros(M)
    avg_vtCSR = np.zeros(M)
    
    CSR_sets = attack_multiple_images(images, 'SIM_csr', data_type, N_max, br, delta) #returns a list in shape len(N_max) x len(images)
    
    for i in range(M):
        _, _, _, _, avg_data = certifyRobustness_pixel(net=starvNet, in_sets=CSR_sets[i], in_datas=images,
            veriMethod='BFS', reachMethod='approx', lp_solver='gurobi', pool=None, 
            RF=0.0, DR=0, return_output=False, show=False)
        avg_numRbCSR[i], avg_numUnkCSR[i], avg_numMisCSR[i], avg_numAttCSR[i], avg_riouCSR[i], avg_rvCSR[i], avg_rsCSR[i], avg_vtCSR[i] = avg_data
        
        
    print(f"\nVerifying {net_type} SSNN with Sparse Image Star in COO format")
    avg_numRbCOO = np.zeros(M)
    avg_numUnkCOO = np.zeros(M)
    avg_numMisCOO = np.zeros(M)
    avg_numAttCOO = np.zeros(M)
    avg_riouCOO = np.zeros(M)
    avg_rvCOO = np.zeros(M)
    avg_rsCOO = np.zeros(M)
    avg_vtCOO = np.zeros(M)
    
    COO_sets = attack_multiple_images(images, 'SIM_coo', data_type, N_max, br, delta) #returns a list in shape len(N_max) x len(images)
    
    for i in range(M):
        _, _, _, _, avg_data = certifyRobustness_pixel(net=starvNet, in_sets=COO_sets[i], in_datas=images,
            veriMethod='BFS', reachMethod='approx', lp_solver='gurobi', pool=None, 
            RF=0.0, DR=0, return_output=False, show=False)
        avg_numRbCOO[i], avg_numUnkCOO[i], avg_numMisCOO[i], avg_numAttCOO[i], avg_riouCOO[i], avg_rvCOO[i], avg_rsCOO[i], avg_vtCOO[i] = avg_data
        
    IM_data = [avg_numRbIM, avg_numUnkIM, avg_numMisIM, avg_numAttIM, avg_riouIM, avg_rvIM, avg_rsIM, avg_vtIM]
    CSR_data = [avg_numRbCSR, avg_numUnkCSR, avg_numMisCSR, avg_numAttCSR, avg_riouCSR, avg_rvCSR, avg_rsCSR, avg_vtCSR]
    COO_data = [avg_numRbCOO, avg_numUnkCOO, avg_numMisCOO, avg_numAttCOO, avg_riouCOO, avg_rvCOO, avg_rsCOO, avg_vtCOO]
    
    save_file = path + f"/{net_type}SSNN_M2NIST_CAV2021_results.pkl"
    pickle.dump([IM_data, CSR_data, COO_data], open(save_file, "wb"))


    print('=====================================================')
    print('DONE!')
    print('=====================================================')
    
    
# def verify_CAV2021_MNIST_SSNN(net_type, dtype='float64'):
#     print('=========================================================================================')
#     print(f"Verification of CAV2021 MNIST Semantic Segmentation Neural Netowrk against Pixel Attacks")
#     print('=========================================================================================\n')
    
#     data_type = 'mnist'
#     folder_dir = f"./SparseImageStar_evaluation/CAV2021_SSNN/MNIST/"
#     starvNet = load_network(folder_dir, net_type, data_type, dtype)
    
#     print(starvNet.info())
#     data, labels = load_dataset(folder_dir, data_type)
#     N = 2 #20 # number of tested images
#     labels = labels[:N]
#     images = [data[:, :, i] for i in range(N)]
    
# #     N_max = [10, 20, 30, 40, 50]
#     N_max = [10]
#     M = len(N_max)
#     br = 150
#     delta = 0.001
#     num_pixels = np.prod(data[:, :, 0].shape) 
    
#     UNK_PIX = 10 # unknown pixels; pixels which have multiple classification labels
#     MIS_PIX = 11 # not robust pixels; miss classified pixels
    
    
#     print(f"Verifying {net_type} SSNN with ImageStar")
#     avg_numRbIM = np.zeros(M)
#     avg_numUnkIM = np.zeros(M)
#     avg_numMisIM = np.zeros(M)
#     avg_numAttIM = np.zeros(M)
#     avg_riouIM = np.zeros(M)
#     avg_rvIM = np.zeros(M)
#     avg_rsIM = np.zeros(M)
#     avg_vtIM = np.zeros(M)
    
#     IM_sets = attack_multiple_images(images, 'IM', data_type, N_max, br, delta) #returns a list in shape len(N_max) x len(images)
    
#     for i in range(M):
#         veriIM, vtIM, _, _, avg_data = certifyRobustness_pixel(net=starvNet, in_sets=IM_sets[i], in_datas=images,
#             veriMethod='BFS', reachMethod='approx', lp_solver='gurobi', pool=None, 
#             RF=0.0, DR=0, return_output=False, show=True)
#         avg_numRbIM[i], avg_numUnkIM[i], avg_numMisIM[i], avg_numAttIM[i], avg_riouIM[i], avg_rvIM[i], avg_rsIM[i], avg_vtIM[i] = avg_data
#     del IM_sets

    
#     print(f"\nVerifying {net_type} SSNN with Sparse Image Star in CSR format")
#     avg_numRbCSR = np.zeros(M)
#     avg_numUnkCSR = np.zeros(M)
#     avg_numMisCSR = np.zeros(M)
#     avg_numAttCSR = np.zeros(M)
#     avg_riouCSR = np.zeros(M)
#     avg_rvCSR = np.zeros(M)
#     avg_rsCSR = np.zeros(M)
#     avg_vtCSR = np.zeros(M)
    
#     CSR_sets = attack_multiple_images(images, 'SIM_csr', data_type, N_max, br, delta) #returns a list in shape len(N_max) x len(images)
    
#     for i in range(M):
#         veriCSR, vtCSR, _, _, avg_data = certifyRobustness_pixel(net=starvNet, in_sets=CSR_sets[i], in_datas=images,
#             veriMethod='BFS', reachMethod='approx', lp_solver='gurobi', pool=None, 
#             RF=0.0, DR=0, return_output=False, show=True)
#         avg_numRbCSR[i], avg_numUnkCSR[i], avg_numMisCSR[i], avg_numAttCSR[i], avg_riouCSR[i], avg_rvCSR[i], avg_rsCSR[i], avg_vtCSR[i] = avg_data
#     del CSR_sets
        
        
#     print(f"\nVerifying {net_type} SSNN with Sparse Image Star in COO format")
#     avg_numRbCOO = np.zeros(M)
#     avg_numUnkCOO = np.zeros(M)
#     avg_numMisCOO = np.zeros(M)
#     avg_numAttCOO = np.zeros(M)
#     avg_riouCOO = np.zeros(M)
#     avg_rvCOO = np.zeros(M)
#     avg_rsCOO = np.zeros(M)
#     avg_vtCOO = np.zeros(M)
    
#     COO_sets = attack_multiple_images(images, 'SIM_coo', data_type, N_max, br, delta) #returns a list in shape len(N_max) x len(images)
    
#     for i in range(M):
#         veriCOO, vtCOO, _, _, avg_data = certifyRobustness_pixel(net=starvNet, in_sets=COO_sets[i], in_datas=images,
#             veriMethod='BFS', reachMethod='approx', lp_solver='gurobi', pool=None, 
#             RF=0.0, DR=0, return_output=False, show=True)
#         avg_numRbCOO[i], avg_numUnkCOO[i], avg_numMisCOO[i], avg_numAttCOO[i], avg_riouCOO[i], avg_rvCOO[i], avg_rsCOO[i], avg_vtCOO[i] = avg_data
        
        
        
        

    
#     print(f"\nVerifying {net_type} SSNN with Sparse Image Star in CSR format")
#     avg_numRbIM = np.zeros(M)
#     avg_numUnkIM = np.zeros(M)
#     avg_numMisIM = np.zeros(M)
#     avg_numAttIM = np.zeros(M)
#     avg_riouIM = np.zeros(M)
#     avg_rvIM = np.zeros(M)
#     avg_rsIM = np.zeros(M)
#     avg_vtIM = np.zeros(M)
    
#     CSR_sets = attack_multiple_images(images, 'SIM_csr', data_type, N_max, br, delta) #returns a list in shape len(N_max) x len(images)
    
#     for i in range(M):
#         num_rbPixIM = np.zeros(N)  # number of robust pixels in ImageStar
#         num_unkPixIM = np.zeros(N) # number of unknown pixels in ImageStar
#         num_misPixIM = np.zeros(N) # number of missclassified pixels in ImageStar
#         num_attPixIM = np.zeros(N) # number of attacked pixels in ImageStar
#         iou = np.zeros(N)
        
#         print(f"Verifying netowrk with N_max={N_max}, br={br}, delta={delta}")
        
#         VerIM, vtIM, _, _ = certifyRobustness_pixel(net=starvNet, in_sets=IM_sets[i], in_datas=images,
#             veriMethod='BFS', reachMethod='approx', lp_solver='gurobi', pool=None, 
#             RF=0.0, DR=0, return_output=False, show=True)
        
#         for j in range(N):
#             num_attPixIM[j] = IM_sets[i][j].geNumAttackedPixels()
#             num_misPixIM[j] = (VerIM == MIS_PIX).sum()
#             num_unkPixIM[j] = (VerIM == UNK_PIX).sum()
#             num_rbPixIM[j] = num_pixels - (unrbPixIM[j] + unkPixIM[j])
#             iou[j] = jaccard_score(VerIM.ravel(), images[j].ravel())
        
#         avg_numRbIM[i] = num_rbPixIM.sum() / N
#         avg_numUnkIM[i] = num_unkPixIM.sum() / N
#         avg_numMisIM[i] = num_misPixIM.sum() / N
#         avg_numAttIM[i] = num_attPixIM.sum() / N
#         avg_riouIM[i] = iou.sum() / N
#         avg_rvIM[i] = (num_rbPixIM / num_pixels) / N
#         avg_rsIM[i] = ((num_misPixIM + num_unkPixIM) / num_attPixIM).sum() / N
#         avg_vtIM[i] = vtIM.sum() / N
        
    
    
#     print(f"\nVerifying {net_type} SSNN with Sparse Image Star in CSR format")
#     avg_numRbCSR = np.zeros(M)
#     avg_numUnkCSR = np.zeros(M)
#     avg_numMisCSR = np.zeros(M)
#     avg_numAttCSR = np.zeros(M)
#     avg_riouCSR = np.zeros(M)
#     avg_rvCSR = np.zeros(M)
#     avg_rsCSR = np.zeros(M)
#     avg_vtCSR= np.zeros(M)
#     CSR_sets = attack_multiple_images(images, 'SIM_csr', data_type, N_max, br, delta) #returns a list in shape len(N_max) x len(images)
    
#     for i in range(M):
#         num_rbPixCSR = np.zeros(N)  # number of robust pixels in ImageStar
#         num_unkPixCSR = np.zeros(N) # number of unknown pixels in ImageStar
#         num_misPixCSR = np.zeros(N) # number of missclassified pixels in ImageStar
#         num_attPixCSR = np.zeros(N) # number of attacked pixels in ImageStar
#         iou = np.zeros(N)
        
#         print(f"Verifying netowrk with N_max={N_max}, br={br}, delta={delta}")
        
#         VerCSR vtCSR, _, _ = certifyRobustness_pixel(net=starvNet, in_sets=CSR_sets[i], in_datas=images,
#             veriMethod='BFS', reachMethod='approx', lp_solver='gurobi', pool=None, 
#             RF=0.0, DR=0, return_output=False, show=True)
        
#         for j in range(N):
#             num_attPixIM[j] = CSR_sets[i][j].geNumAttackedPixels()
#             num_misPixIM[j] = (VerIM == MIS_PIX).sum()
#             num_unkPixIM[j] = (VerIM == UNK_PIX).sum()
#             num_rbPixIM[j] = num_pixels - (unrbPixIM[j] + unkPixIM[j])
#             iou[j] = jaccard_score(VerIM.ravel(), images[j].ravel())
    
#         avg_numRbCSR[i] = num_rbPixIM.sum() / N
#         avg_numUnkCSR[i] = num_unkPixIM.sum() / N
#         avg_numMisCSR[i] = num_misPixIM.sum() / N
#         avg_numAttCSR[i] = num_attPixIM.sum() / N
#         avg_riouCSR[i] = iou.sum() / N
#         avg_rvCSR[i] = (num_rbPixIM / num_pixels) / N
#         avg_rsCSR[i] = ((num_misPixIM + num_unkPixIM) / num_attPixIM).sum() / N
#         avg_vtCSR[i] = vtIM.sum() / N
        
        
    
#     print(f"\nVerifying {net_type} SSNN with Sparse Image Star in COO format")
#     avg_numRbCOO = np.zeros(M)
#     avg_numUnkCOO = np.zeros(M)
#     avg_numMisCOO = np.zeros(M)
#     avg_numAttCOO = np.zeros(M)
#     avg_riouCOO = np.zeros(M)
#     avg_rvCOO = np.zeros(M)
#     avg_rsCOO = np.zeros(M)
#     avg_vtCOO = np.zeros(M)
#     COO_sets = attack_multiple_images(images, 'SIM_coo', data_type, N_max, br, delta) #returns a list in shape len(N_max) x len(images)
    
#     for i in range(M):
#         num_rbPixCOO = np.zeros(N)  # number of robust pixels in ImageStar
#         num_unkPixCOO = np.zeros(N) # number of unknown pixels in ImageStar
#         num_misPixCOO = np.zeros(N) # number of missclassified pixels in ImageStar
#         num_attPixCOO = np.zeros(N) # number of attacked pixels in ImageStar
#         iou = np.zeros(N)
        
#         print(f"Verifying netowrk with N_max={N_max}, br={br}, delta={delta}")
        
#         VerIM, vtIM, _, _ = certifyRobustness_pixel(net=starvNet, in_sets=IM_sets[i], in_datas=images,
#             veriMethod='BFS', reachMethod='approx', lp_solver='gurobi', pool=None, 
#             RF=0.0, DR=0, return_output=False, show=True)
        
#         for j in range(N):
#             num_attPixIM[j] = IM_sets[i][j].geNumAttackedPixels()
#             num_misPixIM[j] = (VerIM == MIS_PIX).sum()
#             num_unkPixIM[j] = (VerIM == UNK_PIX).sum()
#             num_rbPixIM[j] = num_pixels - (num_misPixIM[j] + num_unkPixIM[j])
#             iou[j] = jaccard_score(VerIM.ravel(), images[j].ravel())
    
#         avg_numRbCSR[i] = num_rbPixIM.sum() / N
#         avg_numUnkCSR[i] = num_unkPixIM.sum() / N
#         avg_numMisCSR[i] = num_misPixIM.sum() / N
#         avg_numAttCSR[i] = num_attPixIM.sum() / N
#         avg_riouCSR[i] = iou.sum() / N
#         avg_rvCSR[i] = (num_rbPixIM / num_pixels) / N
#         avg_rsCSR[i] = ((num_misPixIM + num_unkPixIM) / num_attPixIM).sum() / N
#         avg_vtCSR[i] = vtIM.sum() / N

#     print('=====================================================')
#     print('DONE!')
#     print('=====================================================')
    
    
# def verify_CAV2021_M2NIST_SSNN(net_type, dtype='float64'):
#     print('==========================================================================================')
#     print(f"Verification of CAV2021 M2NIST Semantic Segmentation Neural Netowrk against Pixel Attacks")
#     print('==========================================================================================\n')
    
#     data_type = 'm2nist'
#     folder_dir = f"./SparseImageStar_evaluation/CAV2021_SSNN/M2NIST/"
#     starvNet = load_network(folder_dir, net_type, data_type, dtype)
    
#     print(starvNet.info())
#     data, labels = load_dataset(folder_dir, data_type)
#     N = 20 # number of tested images
# #     images = data[:, :, :N]
#     labels = labels[:N]
#     images = [data[:, :, i] for i in range(N)]
    
#     N_max = [5, 10, 15, 20, 25]
#     M = len(N_max)
#     br = 150
#     delta = 0.00001
    
#     rbIM = np.zeros([M, N])
#     vtIM = np.zeros([M, N])
#     rbCSR = np.zeros([M, N])
#     vtCSR = np.zeros([M, N])
#     rbCOO = np.zeros([M, N])
#     vtCOO = np.zeros([M, N])
    
#     print('rbIM: ', rbIM.shape)
#     print('vtIM: ', vtIM.shape)
    
#     print(f"Verifying {net_type} SSNN with ImageStar")
#     rbIM_table = []
#     vtIM_table = []
#     IM_sets = attack_multiple_images(images, 'IM', data_type, N_max, br, delta)
#     for i in len(N_max):
#         rb_delta_table = ['ImageStar']
#         vt_delta_table = ['ImageStar']
#         print(f"Verifying netowrk with N_max={N_max}, br={br}, delta={delta}")

#         rbIM[i, :], vtIM[i, :], _, _ = certifyRobustness_pixel(net=starvNet, in_sets=IM_sets[i], in_datas=images,
#             veriMethod='BFS', reachMethod='approx', lp_solver='gurobi', pool=None, 
#             RF=0.0, DR=0, return_output=False, show=False)
#         rbIM_table.append((rbIM[i, :]==1).sum())
#         vtIM_table.append((vtIM[i, :].sum() / N))
    
#     print(f"\nVerifying {net_type} ConvNet with Sparse Image Star in CSR format")
#     rbCSR_table = []
#     vtCSR_table = []
#     CSR_sets = attack_multiple_images(images, 'SIM_csr', data_type, N_max, br, delta)
#     for i in range(M):
#         rb_delta_table = ['SIM_CSR']
#         vt_delta_table = ['SIM_CSR']
#         print(f"Verifying netowrk with N_max={N_max}, br={br}, delta={delta}")

#         rbCSR[i, :], vtCSR[i, :], _, _ = certifyRobustness_pixel(net=starvNet, in_sets=CSRs[i], in_datas=images,
#                 veriMethod='BFS', reachMethod='approx', lp_solver='gurobi', pool=None, 
#                 RF=0.0, DR=0, return_output=True, show=False)
#         rbCSR_table.append((rbCSR[i, :]==1).sum())
#         vtCSR_table.append((vtCSR[i, :].sum() / N))
            
#     print(f"\nVerifying {net_type} ConvNet with Sparse Image Star in COO format")
#     rbCOO_table = []
#     vtCOO_table = []
#     COO_sets = attack_multiple_images(images, 'SIM_coo', data_type, N_max, br, delta)
#     for i in range(M):
#         rb_delta_table = ['SIM_COO']
#         vt_delta_table = ['SIM_COO']
#         print(f"Verifying netowrk with N_max={N_max}, br={br}, delta={delta}")

#         rbCOO[i, :], vtCOO[i, :], _, _ = certifyRobustness_pixel(net=starvNet, in_sets=COOs[i], in_datas=images,
#                 veriMethod='BFS', reachMethod='approx', lp_solver='gurobi', pool=None, 
#                 RF=0.0, DR=0, return_output=True, show=False)
#         rbCOO_table.append((rbCOO[i, :]==1).sum())
#         vtCOO_table.append((vtCOO[i, :].sum() / N))
        
        
#     # save verification results
#     path = f"./SparseImageStar_evaluation/results"
#     if not os.path.exists(path):
#         os.makedirs(path)
    
#     if net_type == 'dilated':
#         file_name = path + f"{folder_dir}/m2nist_{net_type}_72iou_24layer_pixelAttack_resluts"
#     elif net_type == 'dilatedcnn_avgpool':
#         file_name = path + f"{folder_dir}/m2nist_62iou_{net_type}_pixelAttack_resluts"
#     else:
#         file_name = path + f"{folder_dir}/m2nist_75iou_{net_type}_pixelAttack_resluts"
    
#     headers = []
#     for n_max in N_max:
#         headers.append(f"N_max={n_max}")
    
#     # Robustness Resluts
#     data = [rbIM_table, rbCSR_table, rbCOO_table]
#     print('-----------------------------------------------------')
#     print('Robustness')
#     print('-----------------------------------------------------')
#     print(tabulate(data, headers=headers))
#     print()

#     Tlatex = tabulate(data, headers=headers, tablefmt='latex')
#     with open(file_name + "_rb.tex", "w") as f:
#         print(Tlatex, file=f)

#     # Verification Time Results
#     data = [vtIM_table, vtCSR_table, vtCOO_table]
#     print('-----------------------------------------------------')
#     print('Verification Time')
#     print('-----------------------------------------------------')
#     print(tabulate(data, headers=headers))
#     print()

#     Tlatex = tabulate(data, headers=headers, tablefmt='latex')
#     with open(file_name + "_vt.tex", "w") as f:
#         print(Tlatex, file=f)

#     pickle.dump([rbIM, vtIM, rbIM_table, vtIM_table, rbCSR, vtCSR, rbCSR_table, vtCSR_table, rbCOO, vtCOO, rbCOO_table, vtCOO_table], open(file_name, "wb"))

#     print('=====================================================')
#     print('DONE!')
#     print('=====================================================')
    
# if __name__ == "__main__":
verify_CAV2021_MNIST_SSNN(net_type='relu', dtype='float64')
# verify_CAV2021_MNIST_SSNN(net_type='relu_maxpool', dtype='float64')
# verify_CAV2021_MNIST_SSNN(net_type='dilated', dtype='float64')
# verify_CAV2021_M2NIST_SSNN(net_type='dilatedcnn_avgpool', dtype='float64')
# verify_CAV2021_M2NIST_SSNN(net_type='transposedcnn_avgpool', dtype='float64')
# verify_CAV2021_M2NIST_SSNN(net_type='dilated', dtype='float64')

Verification of CAV2021 MNIST Semantic Segmentation Neural Netowrk against Pixel Attacks


=============NETWORK===============
Network type: None
Input Dimension: 1
Output Dimension: 1
Number of Layers: 20
Layer types:
Layer 0: <class 'StarV.layer.FullyConnectedLayer.FullyConnectedLayer'> (1, 1, dtype=float64)
Layer 1: <class 'StarV.layer.Conv2DLayer.Conv2DLayer'> (1, 128, kernel_size = (3, 3), stride = [1 1], padding = [1 1], dtype=float64)
Layer 2: <class 'StarV.layer.Conv2DLayer.Conv2DLayer'> (128, 128, kernel_size = (3, 3), stride = [1 1], padding = [1 1], dtype=float64)
Layer 3: <class 'StarV.layer.ReLULayer.ReLULayer'>
Layer 4: <class 'StarV.layer.BatchNorm2DLayer.BatchNorm2DLayer'> (128, eps=9.999999747378752e-06, dtype=float64)
Layer 5: <class 'StarV.layer.AvgPool2DLayer.AvgPool2DLayer'> (kernel_size = [2 2], stride = [2 2], padding = [0 0])
Layer 6: <class 'StarV.layer.Conv2DLayer.Conv2DLayer'> (128, 256, kernel_size = (3, 3), stride = [1 1], padding = [1 1], dtype=float64)
La


KeyboardInterrupt



In [ ]:
net_type='relu'; dtype='float64'
data_type = 'mnist'
folder_dir = f"./SparseImageStar_evaluation/CAV2021_SSNN/MNIST/"
starvNet = load_network(folder_dir, net_type, data_type, dtype)

print(starvNet.info())
data, labels = load_dataset(folder_dir, data_type)
N = 20 # number of tested images
images = [data[:, :, i] for i in range(N)]

N_max = [10, 20, 30, 40, 50]
M = len(N_max)
br = 150
delta = 0.001

rbIM = np.zeros([M, N])
vtIM = np.zeros([M, N])
rbCSR = np.zeros([M, N])
vtCSR = np.zeros([M, N])
rbCOO = np.zeros([M, N])
vtCOO = np.zeros([M, N])

print(f"Verifying {net_type} SSNN with ImageStar")
rbIM_table = []
vtIM_table = []
IM_sets = attack_multiple_images(images, 'IM', data_type, N_max, br, delta) 

image = images[0]
if data_type == 'mnist':
    h = image.shape[0] - 1
    w = image.shape[1] - 1
else:
    h, w = 63, 83

attack_image = image.copy() 
cnt = 0
for i in range(h):
    for j in range(w):
        if image[i, j] > br:
            attack_image[i, j] = 0
            cnt += 1
            if cnt == N_max:
                break
    else:
         continue
    break

noise = attack_image - image
V = np.concatenate([image[:, :, None, None], noise[:, :, None, None]], axis=3)
# V = np.stack([image, noise], axis=)
C = np.array([[1], [-1]])
d = np.array([1, delta-1])
IM = ImageStar(V, C, d, np.array([1-delta]), np.array([1]))
# if format == 'IM':
#     return IM
# elif format == 'SIM_coo':
#     return IM.toSIM('coo')
# else:
#     return IM.toSIM('csr')

In [ ]:
IM_sets[0][0].geNumAttackedPixels()

In [ ]:
repr(IM)

In [ ]:
IM.V[:, :, 0, 0]

In [ ]:
IM.V[:, :, 0, 1]

In [3]:
net_type='relu'; dtype='float64'
data_type = 'mnist'
folder_dir = f"./SparseImageStar_evaluation/CAV2021_SSNN/MNIST/"
starvNet = load_network(folder_dir, net_type, data_type, dtype)

print(starvNet.info())
data, labels = load_dataset(folder_dir, data_type)
N = 20 # number of tested images
images = data[:, :, :N]
labels = labels[:N]

N_max = [10, 20, 30, 40, 50]
M = len(N_max)
br = 150
delta = 0.001

rbIM = np.zeros([M, N])
vtIM = np.zeros([M, N])
rbCSR = np.zeros([M, N])
vtCSR = np.zeros([M, N])
rbCOO = np.zeros([M, N])
vtCOO = np.zeros([M, N])

print(f"Verifying {net_type} SSNN with ImageStar")
rbIM_table = []
vtIM_table = []
IM_sets = attack_multiple_images(images, 'IM', data_type, N_max, br, delta) #returns a list in shape len(N_max) x len(images)


=============NETWORK===============
Network type: None
Input Dimension: 1
Output Dimension: 1
Number of Layers: 20
Layer types:
Layer 0: <class 'StarV.layer.FullyConnectedLayer.FullyConnectedLayer'> (1, 1, dtype=float64)
Layer 1: <class 'StarV.layer.Conv2DLayer.Conv2DLayer'> (1, 128, kernel_size = (3, 3), stride = [1 1], padding = [1 1], dtype=float64)
Layer 2: <class 'StarV.layer.Conv2DLayer.Conv2DLayer'> (128, 128, kernel_size = (3, 3), stride = [1 1], padding = [1 1], dtype=float64)
Layer 3: <class 'StarV.layer.ReLULayer.ReLULayer'>
Layer 4: <class 'StarV.layer.BatchNorm2DLayer.BatchNorm2DLayer'> (128, eps=9.999999747378752e-06, dtype=float64)
Layer 5: <class 'StarV.layer.AvgPool2DLayer.AvgPool2DLayer'> (kernel_size = [2 2], stride = [2 2], padding = [0 0])
Layer 6: <class 'StarV.layer.Conv2DLayer.Conv2DLayer'> (128, 256, kernel_size = (3, 3), stride = [1 1], padding = [1 1], dtype=float64)
Layer 7: <class 'StarV.layer.Conv2DLayer.Conv2DLayer'> (256, 256, kernel_size = (3, 3), stri

In [4]:
print('=========================================================================================')
print(f"Verification of CAV2021 MNIST Semantic Segmentation Neural Netowrk against Pixel Attacks")
print('=========================================================================================\n')

data_type = 'mnist'
folder_dir = f"./SparseImageStar_evaluation/CAV2021_SSNN/MNIST/"
c = load_network(folder_dir, net_type, data_type, dtype)

print(starvNet.info())
data, labels = load_dataset(folder_dir, data_type)
N = 2 #20 # number of tested images
#     images = data[:, :, :N]
labels = labels[:N]
images = [data[:, :, i] for i in range(N)]

#     N_max = [10, 20, 30, 40, 50]
N_max = [10]
M = len(N_max)
br = 150
delta = 0.001

rbPixIM = [None for _ in range(N)]
vtIM = np.zeros([M, N])
rbPixCSR = [None for _ in range(N)]
vtCSR = np.zeros([M, N])
rbPixCOO =  [None for _ in range(N)]
vtCOO = np.zeros([M, N])

print(f"Verifying {net_type} SSNN with ImageStar")
rbIM_table = []
vtIM_table = []
IM_sets = attack_multiple_images(images, 'IM', data_type, N_max, br, delta) #returns a list in shape len(N_max) x len(images)

print('IM_sets: ', len(IM_sets))
print('IM_sets 2d: ', len(IM_sets[0]))
print('images: ', len(images))
for i in range(M):
    rb_delta_table = ['ImageStar']
    vt_delta_table = ['ImageStar']
    print(f"Verifying netowrk with N_max={N_max}, br={br}, delta={delta}")
    rbPixIM[i], vtIM[i, :], _, _ = certifyRobustness_pixel(net=starvNet, in_sets=IM_sets[i], in_datas=images,
        veriMethod='BFS', reachMethod='approx', lp_solver='gurobi', pool=None, 
        RF=0.0, DR=0, return_output=False, show=True)
    print('T: ', len(T))
    rbIM_table.append((rbIM[i, :]==1).sum())
    vtIM_table.append((vtIM[i, :].sum() / N))

Verification of CAV2021 MNIST Semantic Segmentation Neural Netowrk against Pixel Attacks


=============NETWORK===============
Network type: None
Input Dimension: 1
Output Dimension: 1
Number of Layers: 20
Layer types:
Layer 0: <class 'StarV.layer.FullyConnectedLayer.FullyConnectedLayer'> (1, 1, dtype=float64)
Layer 1: <class 'StarV.layer.Conv2DLayer.Conv2DLayer'> (1, 128, kernel_size = (3, 3), stride = [1 1], padding = [1 1], dtype=float64)
Layer 2: <class 'StarV.layer.Conv2DLayer.Conv2DLayer'> (128, 128, kernel_size = (3, 3), stride = [1 1], padding = [1 1], dtype=float64)
Layer 3: <class 'StarV.layer.ReLULayer.ReLULayer'>
Layer 4: <class 'StarV.layer.BatchNorm2DLayer.BatchNorm2DLayer'> (128, eps=9.999999747378752e-06, dtype=float64)
Layer 5: <class 'StarV.layer.AvgPool2DLayer.AvgPool2DLayer'> (kernel_size = [2 2], stride = [2 2], padding = [0 0])
Layer 6: <class 'StarV.layer.Conv2DLayer.Conv2DLayer'> (128, 256, kernel_size = (3, 3), stride = [1 1], padding = [1 1], dtype=float64)
La

TypeError: certifyRobustness_pixel() missing 1 required positional argument: 'num_classes'

In [ ]:
from StarV.verifier.certifier import *

UNK_PIX = 10 # unknown pixels; pixels which have multiple classification labels
MIS_PIX = 11 # not robust pixels; miss classified pixels

print('=========================================================================================')
print(f"Verification of CAV2021 MNIST Semantic Segmentation Neural Netowrk against Pixel Attacks")
print('=========================================================================================\n')

data_type = 'mnist'
folder_dir = f"./SparseImageStar_evaluation/CAV2021_SSNN/MNIST/"
c = load_network(folder_dir, net_type, data_type, dtype)

print(starvNet.info())
data, labels = load_dataset(folder_dir, data_type)
N = 2 #20 # number of tested images
#     images = data[:, :, :N]
labels = labels[:N]
images = [data[:, :, i] for i in range(N)]

#     N_max = [10, 20, 30, 40, 50]
N_max = [10]
M = len(N_max)
br = 150
delta = 0.001

print(f"Verifying {net_type} SSNN with ImageStar")
avg_numRbIM = np.zeros(M)
avg_numUnkIM = np.zeros(M)
avg_numMisIM = np.zeros(M)
avg_numAttIM = np.zeros(M)
avg_riouIM = np.zeros(M)
avg_rvIM = np.zeros(M)
avg_rsIM = np.zeros(M)
avg_vtIM = np.zeros(M)

IM_sets = attack_multiple_images(images, 'IM', data_type, N_max, br, delta) #returns a list in shape len(N_max) x len(images)

# for i in range(M):
i = 0
num_rbPix  = np.zeros(N) # number of robust pixels
num_unkPix = np.zeros(N) # number of unknown pixels
num_misPix = np.zeros(N) # number of missclassified pixels
num_attPix = np.zeros(N) # number of attacked pixels
iou = np.zeros(N)

num_pixels = np.prod(images[0].shape)

# for j in range(N):
j = 1
veri_image, veri_time[i], _, O = certifyPixelRobustness_single_input(starvNet, IM_sets[i][j], images[j],
     veriMethod='BFS', reachMethod='approx', lp_solver='gurobi', pool=None, 
     RF=0.0, DR=0, show=True)
veri_set.append(veri_image)

num_attPix[j] = IM_sets[i][j].geNumAttackedPixels()
num_misPix[j] = (veri_image == MIS_PIX).sum()
num_unkPix[j] = (veri_image == UNK_PIX).sum()
num_rbPix[j] = num_pixels - (num_misPix[j] + num_unkPix[j])
iou[j] = jaccard_score(veri_image.ravel(), images[i].ravel())

avg_numRb = num_rbPix.sum() / N
avg_numUnk = num_unkPix.sum() / N
avg_numMis = num_misPix.sum() / N
avg_numAtt = num_attPix.sum() / N
avg_riou = iou.sum() / N
avg_rv = (num_rbPix / num_pixels) / N
avg_rs = (num_misPix + num_unkPix / num_attPix).sum() / N
avg_vt = veri_time.sum() / N
avg_data = [avg_numRb, avg_numUnk, avg_numMis, avg_numAtt, avg_riou, avg_rv, avg_rs, avg_vt]

Verification of CAV2021 MNIST Semantic Segmentation Neural Netowrk against Pixel Attacks


=============NETWORK===============
Network type: None
Input Dimension: 1
Output Dimension: 1
Number of Layers: 20
Layer types:
Layer 0: <class 'StarV.layer.FullyConnectedLayer.FullyConnectedLayer'> (1, 1, dtype=float64)
Layer 1: <class 'StarV.layer.Conv2DLayer.Conv2DLayer'> (1, 128, kernel_size = (3, 3), stride = [1 1], padding = [1 1], dtype=float64)
Layer 2: <class 'StarV.layer.Conv2DLayer.Conv2DLayer'> (128, 128, kernel_size = (3, 3), stride = [1 1], padding = [1 1], dtype=float64)
Layer 3: <class 'StarV.layer.ReLULayer.ReLULayer'>
Layer 4: <class 'StarV.layer.BatchNorm2DLayer.BatchNorm2DLayer'> (128, eps=9.999999747378752e-06, dtype=float64)
Layer 5: <class 'StarV.layer.AvgPool2DLayer.AvgPool2DLayer'> (kernel_size = [2 2], stride = [2 2], padding = [0 0])
Layer 6: <class 'StarV.layer.Conv2DLayer.Conv2DLayer'> (128, 256, kernel_size = (3, 3), stride = [1 1], padding = [1 1], dtype=float64)
La

In [ ]:
len(IM_sets[0])

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from tabulate import tabulate
import pickle

import os
import sys
sys.path.append(os.path.abspath(os.curdir))

from StarV.set.imagestar import ImageStar
from StarV.set.sparseimagestar2dcoo import SparseImageStar2DCOO
from StarV.set.sparseimagestar2dcsr import SparseImageStar2DCSR
from StarV.verifier.certifier import certifyRobustness_pixel
from StarV.util.load import *
from sklearn.metrics import jaccard_score


from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

np.set_printoptions(edgeitems=30, linewidth=100000)

data_type = 'mnist'
folder_dir = f"./SparseImageStar_evaluation/CAV2021_SSNN/MNIST"
starvNet = load_network(folder_dir, net_type, data_type, dtype)

print(starvNet.info())
data, labels = load_dataset(folder_dir, data_type)
N = 20 # number of tested images
labels = labels[:N]
images = [data[:, :, i] for i in range(N)]

In [ ]:
gr_pix_id = starvNet.evaluate(images[0]).squeeze(axis=2)

In [ ]:
y[:, :, 0]

In [ ]:
y.shape

In [ ]:
IM_sets[0][0].classes is None

In [ ]:
a = np.random.rand(4, 4)
b = np.random.rand(4, 4)
c = np.concatenate([a[:, :, None], b[:, :, None]], axis=2)
print(c.shape)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate
import torchvision.transforms as transforms
import pickle
import re
from timeit import default_timer as timer

import os
import sys
sys.path.append(os.path.abspath(os.curdir))

from StarV.set.imagestar import ImageStar
from StarV.set.sparseimagestar2dcoo import SparseImageStar2DCOO
from StarV.set.sparseimagestar2dcsr import SparseImageStar2DCSR
from StarV.verifier.certifier import certifyRobustness
from StarV.util.load import *
from StarV.util.vnnlib import *

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

np.set_printoptions(edgeitems=30, linewidth=100000)

def load_cav2021_network(net_dir, net_type, dtype='float64'):

    assert net_type in ['relu', 'relu_maxpool', 'dilated'], \
    f"network type for CAV2021 networks should 'relu', 'relu_maxpool', or 'dilated' but received {net_type}"
    # loading DNNs into StarV network
    network = load_neural_network_file(net_dir, dtype=dtype, channel_last=True, show=True)
    return network

def load_cav2021_dataset(data_type, folder_dir, dtype='float64'):
    data_dir = f"{folder_dir}/{data_type.lower()}_test.csv" 
    tests = np.genfromtxt(data_dir, delimiter=',', dtype=dtype)
    N = len(tests)
    return tests, N

def normalize(image, data_type):
    img = image.copy()
    if data_type == 'CIFAR10':
        mean = np.array([0.4914, 0.4822, 0.4465])
        std = np.array([0.2023, 0.1994, 0.2010])
        img -= mean
        img /= std
    return img

def verify_cav2021_network(net_type='relu', dtype='float64'):
    print('=================================================================================')
    print(f"Verification of CAV2021 {net_type} MNIST Network")
    print('=================================================================================\n')
    
    folder_dir = f"./SparseImageStar_evaluation/CAV2021_SSNN/MNIST/"
    if net_type == 'dilated':
        net_dir = f"{folder_dir}/mnist_{net_type}_net)_21_later_83iou.onnx"
    else:
        net_dir = f"{folder_dir}/net_mnist_3_{net_type}.onnx"
#     starvNet = load_neural_network_file(net_dir, dtype=dtype)
#     print()
#     print(starvNet.info())
    
    model = onnx.load(net_dir)
    return model
    
import onnx, onnx2pytorch
from onnx2torch import convert

In [ ]:
from StarV.util.load import *
def load_convnet(net_dir, net_type, dtype='float32'):

    assert net_type in ['Small', 'Medium', 'Large'], \
    f"There are 3 types of ConvNet networks: /'Small/', /'Medium/', and /'Large/'"

    if net_type == 'Small':
        network = load_CAV2020_MNIST_Small_ConvNet(net_dir=net_dir, dtype=dtype)
    elif net_type == 'Medium':
        network = load_CAV2020_MNIST_Medium_ConvNet(net_dir=net_dir, dtype=dtype)
    elif net_type == 'Large':
        network = load_CAV2020_MNIST_Large_ConvNet(net_dir=net_dir, dtype=dtype)
    else:
        raise Exception('Unknown network type for ConvNet')
    return network

dtype='float64'
net_type='Small'
folder_dir = f"./SparseImageStar_evaluation/CAV2020_ImageStar/MNIST_NETS/{net_type}"
net_dir = f"{folder_dir}/{net_type}_ConvNet.onnx"
# starvNet = load_convnet(net_dir, net_type, dtype=dtype)
model = onnx.load(net_dir)


# model is an onnx model
graph = model.graph
# graph inputs
for input_name in graph.input:
    print(input_name)
# # graph parameters
# for init in graph.init:
#     print(init.name)
# graph outputs
for output_name in graph.output:
    print(output_name)
# iterate over nodes
for node in graph.node:
    # node inputs
    for idx, node_input_name in enumerate(node.input):
        print(idx, node_input_name)
    # node outputs
    for idx, node_output_name in enumerate(node.output):
        print(idx, node_output_name)

In [ ]:
model = verify_cav2021_network('relu')
# model = convert(model)
model = onnx2pytorch.ConvertModel(model)
onnx.checker.check_model(model)

In [ ]:
# model is an onnx model
graph = model.graph
# graph inputs
for input_name in graph.input:
    print(input_name)
# # graph parameters
# for init in graph.init:
#     print(init.name)
# graph outputs
for output_name in graph.output:
    print(output_name)
# iterate over nodes
for node in graph.node:
    # node inputs
    for idx, node_input_name in enumerate(node.input):
        print(idx, node_input_name)
    # node outputs
    for idx, node_output_name in enumerate(node.output):
        print(idx, node_output_name)

In [ ]:
model = verify_cav2021_network('relu')
model = onnx2pytorch.ConvertModel(model, debug=True)
onnx.checker.check_model(model)